### Import Package and Libraries
- spotipy
- scikit-learn
- tfidfVectorizer
- pandas

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re, os
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

#connect to Spotify Web API
client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                        client_secret=client_secret)
sp = spotipy.client.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
df = pd.read_csv('processed_dataset.csv')
df.head(10)

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_uri,uri,track_href,analysis_url,duration_ms_x,time_signature,pos,artist_name,artist_uri,track_name,album_uri,duration_ms_y,album_name,name,artist_pop,genres,track_pop,id
0,0,0.890,0.672,7.0,-5.532,1.0,0.0621,0.70700,0.000604,0.1770,0.5980,126.036,audio_features,66U0ASk1VHZsqIkpMjKX3B,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0,0,AronChupa,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop,59,66U0ASk1VHZsqIkpMjKX3B
1,1,0.883,0.817,0.0,-5.414,1.0,0.2350,0.60100,0.004490,0.1100,0.5950,128.078,audio_features,5MhsZlmKJG6X5kTHkdwC4B,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849.0,4.0,1,AronChupa,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,72,5MhsZlmKJG6X5kTHkdwC4B
2,2,0.586,0.626,1.0,-8.448,1.0,0.0356,0.08740,0.000380,0.1130,0.0519,94.965,audio_features,0GZoB8h0kqXn7XFm4Sj06k,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507.0,4.0,2,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop,0,0GZoB8h0kqXn7XFm4Sj06k
3,3,0.654,0.304,6.0,-9.067,0.0,0.0358,0.64700,0.000072,0.1470,0.1020,113.992,audio_features,35kahykNu00FPysz3C2euR,spotify:track:35kahykNu00FPysz3C2euR,https://api.spotify.com/v1/tracks/35kahykNu00F...,https://api.spotify.com/v1/audio-analysis/35ka...,216600.0,4.0,3,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,58,35kahykNu00FPysz3C2euR
4,4,0.690,0.578,6.0,-7.436,1.0,0.0929,0.16700,0.000000,0.3050,0.4160,99.961,audio_features,3G6hD9B2ZHOsgf4WfNu7X1,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,https://api.spotify.com/v1/tracks/3G6hD9B2ZHOs...,https://api.spotify.com/v1/audio-analysis/3G6h...,193059.0,4.0,4,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,76,3G6hD9B2ZHOsgf4WfNu7X1
5,5,0.500,0.726,5.0,-4.124,1.0,0.0643,0.03620,0.000000,0.4410,0.3970,125.904,audio_features,6WQLkih8nE0JdUCEyLaGnQ,spotify:track:6WQLkih8nE0JdUCEyLaGnQ,https://api.spotify.com/v1/tracks/6WQLkih8nE0J...,https://api.spotify.com/v1/audio-analysis/6WQL...,210013.0,4.0,0,Alesso,spotify:artist:4AVFqumd2ogHFlRbKIjp1t,Heroes (we could be),spotify:album:3yqh9ozOA2cAgPUJaveAuG,210012,Forever,school,75,dance_pop edm electro_house pop pop_dance trop...,0,6WQLkih8nE0JdUCEyLaGnQ
6,6,0.490,0.885,3.0,-4.121,1.0,0.0396,0.00218,0.000000,0.0741,0.6400,166.996,audio_features,37sINbJZcFdHFAsVNsPq1i,spotify:track:37sINbJZcFdHFAsVNsPq1i,https://api.spotify.com/v1/tracks/37sINbJZcFdH...,https://api.spotify.com/v1/audio-analysis/37sI...,245467.0,4.0,1,The Script,spotify:artist:3AQRLZ9PuTAozP28Skbq8V,Superheroes,spotify:album:6yd9yk8nFcHalXzy7mgaDx,245466,No Sound Without Silence,school,73,celtic_rock neo_mellow pop pop_rock,74,37sINbJZcFdHFAsVNsPq1i
7,7,0.383,0.861,4.0,-2.816,0.0,0.0777,0.00380,0.000000,0.1100,0.4800,176.084,audio_features,0yhPEz5KxlDwckGJaMlZqM,spotify:track:0yhPEz5KxlDwckGJaMlZqM,https://api.spotify.com/v1/tracks/0yhPEz5KxlDw...,https://api.spotify.com/v1/audio-analysis/0yhP...,228360.0,4.0,2,Fall Out Boy,spotify:artist:4UXqAaa6dQYAk18Lv7PEgX,Centuries,spotify:album:0dAvbhVYW3bGnLldQ8Nvwn,228360,American Beauty/American Psycho,school,77,emo modern_rock,0,0yhPEz5KxlDwckGJaMlZqM
8,8,0.670,0.905,2.0,-2.385,1.0,0.0339,0.06250,0.000151,0

In [5]:
# Calling Spotify API and Analyzing audio features
audio_features = sp.audio_features(df['track_uri'][0])
audio_features

[{'danceability': 0.89,
  'energy': 0.672,
  'key': 7,
  'loudness': -5.532,
  'mode': 1,
  'speechiness': 0.0621,
  'acousticness': 0.707,
  'instrumentalness': 0.000604,
  'liveness': 0.177,
  'valence': 0.598,
  'tempo': 126.036,
  'type': 'audio_features',
  'id': '66U0ASk1VHZsqIkpMjKX3B',
  'uri': 'spotify:track:66U0ASk1VHZsqIkpMjKX3B',
  'track_href': 'https://api.spotify.com/v1/tracks/66U0ASk1VHZsqIkpMjKX3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/66U0ASk1VHZsqIkpMjKX3B',
  'duration_ms': 163810,
  'time_signature': 4}]

In [6]:
# Calling Spotify API and Analyzing artist features
artist_features = sp.artist(df['artist_uri'][0])
artist_features

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5vCOdeiQt9LyzdI87kt5Sh'},
 'followers': {'href': None, 'total': 384553},
 'genres': ['swedish pop'],
 'href': 'https://api.spotify.com/v1/artists/5vCOdeiQt9LyzdI87kt5Sh',
 'id': '5vCOdeiQt9LyzdI87kt5Sh',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb4ee4d6466fe1d343c0e53b8c',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051744ee4d6466fe1d343c0e53b8c',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1784ee4d6466fe1d343c0e53b8c',
   'width': 160}],
 'name': 'AronChupa',
 'popularity': 60,
 'type': 'artist',
 'uri': 'spotify:artist:5vCOdeiQt9LyzdI87kt5Sh'}

In [7]:
# Calling Spotify API and Analyzing track features
artist = sp.track(df['track_uri'][0])['artists']
artist

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/5vCOdeiQt9LyzdI87kt5Sh'},
  'href': 'https://api.spotify.com/v1/artists/5vCOdeiQt9LyzdI87kt5Sh',
  'id': '5vCOdeiQt9LyzdI87kt5Sh',
  'name': 'AronChupa',
  'type': 'artist',
  'uri': 'spotify:artist:5vCOdeiQt9LyzdI87kt5Sh'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/1KYt3TMGpa1LtVi0m2A0F9'},
  'href': 'https://api.spotify.com/v1/artists/1KYt3TMGpa1LtVi0m2A0F9',
  'id': '1KYt3TMGpa1LtVi0m2A0F9',
  'name': 'Little Sis Nora',
  'type': 'artist',
  'uri': 'spotify:artist:1KYt3TMGpa1LtVi0m2A0F9'}]

In [8]:
df.shape

(81428, 31)

In [9]:
df.drop(columns=["Unnamed: 0"], inplace = True)

In [10]:
# Analysis of the column which are very useful for recommendation
df[['artist_name', 'track_name', 'name', 'artist_pop', 'track_pop', 'genres']].head()

,artist_name,track_name,name,artist_pop,track_pop,genres
0,AronChupa,Little Swing,Party,60,59,swedish_pop
1,AronChupa,I'm an Albatraoz,Party,60,72,swedish_pop
2,Lorde,Yellow Flicker Beat - From The Hunger Games: M...,Party,75,0,art_pop metropopolis nz_pop pop
3,Lorde,White Teeth Teens,Party,75,58,art_pop metropopolis nz_pop pop
4,Lorde,Team,Party,75,76,art_pop metropopolis nz_pop pop


In [11]:
# Creating new attribute to identify same songs with different ids
def drop_duplciate_artist_songs(df):
    df['artists_song'] = df.apply(lambda x: x['artist_name'] + x['track_name'], axis = 1)
    return df.drop_duplicates('artists_song')

song_df = drop_duplciate_artist_songs(df)

def select_features(df):
    return df[['artist_name','id','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
song_df = select_features(song_df)

# preprocess genre
def convert_genre_to_list(df):
    df['genres_list'] = df['genres'].apply(lambda x: x.split(" "))
    return df
song_df = convert_genre_to_list(df)

In [12]:
song_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_uri,uri,track_href,analysis_url,duration_ms_x,time_signature,pos,artist_name,artist_uri,track_name,album_uri,duration_ms_y,album_name,name,artist_pop,genres,track_pop,id,artists_song,genres_list
0,0.890,0.672,7.0,-5.532,1.0,0.0621,0.7070,0.000604,0.177,0.5980,126.036,audio_features,66U0ASk1VHZsqIkpMjKX3B,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0,0,AronChupa,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop,59,66U0ASk1VHZsqIkpMjKX3B,AronChupaLittle Swing,[swedish_pop]
1,0.883,0.817,0.0,-5.414,1.0,0.2350,0.6010,0.004490,0.110,0.5950,128.078,audio_features,5MhsZlmKJG6X5kTHkdwC4B,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849.0,4.0,1,AronChupa,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,72,5MhsZlmKJG6X5kTHkdwC4B,AronChupaI'm an Albatraoz,[swedish_pop]
2,0.586,0.626,1.0,-8.448,1.0,0.0356,0.0874,0.000380,0.113,0.0519,94.965,audio_features,0GZoB8h0kqXn7XFm4Sj06k,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507.0,4.0,2,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop,0,0GZoB8h0kqXn7XFm4Sj06k,LordeYellow Flicker Beat - From The Hunger Gam...,"[art_pop, metropopolis, nz_pop, pop]"
3,0.654,0.304,6.0,-9.067,0.0,0.0358,0.6470,0.000072,0.147,0.1020,113.992,audio_features,35kahykNu00FPysz3C2euR,spotify:track:35kahykNu00FPysz3C2euR,https://api.spotify.com/v1/tracks/35kahykNu00F...,https://api.spotify.com/v1/audio-analysis/35ka...,216600.0,4.0,3,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,58,35kahykNu00FPysz3C2euR,LordeWhite Teeth Teens,"[art_pop, metropopolis, nz_pop, pop]"
4,0.690,0.578,6.0,-7.436,1.0,0.0929,0.1670,0.000000,0.305,0.4160,99.961,audio_features,3G6hD9B2ZHOsgf4WfNu7X1,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,https://api.spotify.com/v1/tracks/3G6hD9B2ZHOs...,https://api.spotify.com/v1/audio-analysis/3G6h...,193059.0,4.0,4,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,76,3G6hD9B2ZHOsgf4WfNu7X1,LordeTeam,"[art_pop, metropopolis, nz_pop, pop]"


### Feature Engineering

#### We will first perform sentiment analysis to generate two new features subjectivity and polarity

Subjectivity (0,1): Subjectivity measures how much factual information and subjective opinion are present in the text. The higher subjectivity means that the text contains personal opinion rather than factual information

Polarity (-1,1): The degree of strong or clearly defined sentiment accounting for negation ( -1 defines a negative sentiment and 1 defines a positive sentiment)

In [13]:
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def get_polarity(text):
    return TextBlob(text).sentiment.polarity

def get_analysis(score, task="polarity"):
    if task == "subjectivity":
        if score < 1/3:
              return "low"
        elif score > 1/3:
              return "high"
        else:
              return "medium"
    else:
        if score < 0:
              return 'Negative'
        elif score == 0:
              return 'Neutral'
        else:
              return 'Positive'

#Adding two features of polarity and subjectivity
def sentiment_analysis(df, text_column):
    df['subjectivity'] = df[text_column].apply(get_subjectivity).apply(lambda x: get_analysis(x, 'subjectivity'))
    df['polarity'] = df[text_column].apply(get_polarity).apply(get_analysis)
    return df

In [14]:
sentiment_song_df = sentiment_analysis(song_df, 'track_name')
sentiment_song_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_uri,uri,track_href,analysis_url,duration_ms_x,time_signature,pos,artist_name,artist_uri,track_name,album_uri,duration_ms_y,album_name,name,artist_pop,genres,track_pop,id,artists_song,genres_list,subjectivity,polarity
0,0.890,0.672,7.0,-5.532,1.0,0.0621,0.7070,0.000604,0.177,0.5980,126.036,audio_features,66U0ASk1VHZsqIkpMjKX3B,spotify:track:66U0ASk1VHZsqIkpMjKX3B,https://api.spotify.com/v1/tracks/66U0ASk1VHZs...,https://api.spotify.com/v1/audio-analysis/66U0...,163810.0,4.0,0,AronChupa,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,Party,60,swedish_pop,59,66U0ASk1VHZsqIkpMjKX3B,AronChupaLittle Swing,[swedish_pop],high,Negative
1,0.883,0.817,0.0,-5.414,1.0,0.2350,0.6010,0.004490,0.110,0.5950,128.078,audio_features,5MhsZlmKJG6X5kTHkdwC4B,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,https://api.spotify.com/v1/tracks/5MhsZlmKJG6X...,https://api.spotify.com/v1/audio-analysis/5Mhs...,166849.0,4.0,1,AronChupa,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,Party,60,swedish_pop,72,5MhsZlmKJG6X5kTHkdwC4B,AronChupaI'm an Albatraoz,[swedish_pop],low,Neutral
2,0.586,0.626,1.0,-8.448,1.0,0.0356,0.0874,0.000380,0.113,0.0519,94.965,audio_features,0GZoB8h0kqXn7XFm4Sj06k,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,https://api.spotify.com/v1/tracks/0GZoB8h0kqXn...,https://api.spotify.com/v1/audio-analysis/0GZo...,232507.0,4.0,2,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,Party,75,art_pop metropopolis nz_pop pop,0,0GZoB8h0kqXn7XFm4Sj06k,LordeYellow Flicker Beat - From The Hunger Gam...,"[art_pop, metropopolis, nz_pop, pop]",low,Neutral
3,0.654,0.304,6.0,-9.067,0.0,0.0358,0.6470,0.000072,0.147,0.1020,113.992,audio_features,35kahykNu00FPysz3C2euR,spotify:track:35kahykNu00FPysz3C2euR,https://api.spotify.com/v1/tracks/35kahykNu00F...,https://api.spotify.com/v1/audio-analysis/35ka...,216600.0,4.0,3,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,58,35kahykNu00FPysz3C2euR,LordeWhite Teeth Teens,"[art_pop, metropopolis, nz_pop, pop]",low,Neutral
4,0.690,0.578,6.0,-7.436,1.0,0.0929,0.1670,0.000000,0.305,0.4160,99.961,audio_features,3G6hD9B2ZHOsgf4WfNu7X1,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,https://api.spotify.com/v1/tracks/3G6hD9B2ZHOs...,https://api.spotify.com/v1/audio-analysis/3G6h...,193059.0,4.0,4,Lorde,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,Party,75,art_pop metropopolis nz_pop pop,76,3G6hD9B2ZHOsgf4WfNu7X1,LordeTeam,"[art_pop, metropopolis, nz_pop, pop]",low,Neutral


#### one hot encoding
One hot encoding is converting categorical data variables into a format that machine learning algorithms can use to make better predictions

In our case we will use one hot encoding to list the sentiment of the song titles

In [15]:
# parameter: (df: dataframe, column: column, name: new column name)
# return: one hot encoded features
def one_hot_encoding(df, column, name): 
    tf_df = pd.get_dummies(df[column])
    features = tf_df.columns
    tf_df.columns = [name + "|" + str(i) for i in features]
    tf_df.reset_index(drop=True, inplace=True)    
    return tf_df

In [16]:
# One-hot encoding for the subjectivity 
ohe_subject = one_hot_encoding(sentiment_song_df, 'subjectivity', 'subject')
ohe_subject.iloc[0]

subject|high      1
subject|low       0
subject|medium    0
Name: 0, dtype: uint8

#### TF-IDF (Term Frequency-Inverse Document Frequency)
Term Frequency-Inverse Document Frequency, often known as TF-IDF, is a technique for counting the number of words in a group of texts. The objective of TF-IDF is to highlight a word's significance within the corpus and documents. The standard formula for TF-IDF calculation is: (Term Frequency x Inverse Document Frequency)

Songs serve as an analogy for the texts in this project. In order to calculate the weight of the genre, we are counting the most prevalent genres inside each song and across all songs. Given that there are no weights used to evaluate the significance and popularity of each genre, which could otherwise result in an overemphasis on less common genres, this is far better than one-hot encoding

In [17]:
# TF-IDF implementation
tf_idf = TfidfVectorizer()
tf_idf_matrix =  tf_idf.fit_transform(song_df['genres_list'].apply(lambda x: " ".join(x)))
df_genre = pd.DataFrame(tf_idf_matrix.toarray())
df_genre.columns = ['genre' + "|" + i for i in tf_idf.get_feature_names_out()]
df_genre.drop(columns='genre|unknown')
df_genre.reset_index(drop = True, inplace=True)

In [18]:
df_genre.iloc[0]

genre|21st_century_classical             0.0
genre|432hz                              0.0
genre|5th_wave_emo                       0.0
genre|_hip_hop                           0.0
genre|_house                             0.0
genre|_roll                              0.0
genre|a_cappella                         0.0
genre|aarhus_indie                       0.0
genre|abstract                           0.0
genre|abstract_beats                     0.0
genre|abstract_hip_hop                   0.0
genre|abstract_idm                       0.0
genre|accordion                          0.0
genre|acid_house                         0.0
genre|acid_jazz                          0.0
genre|acid_rock                          0.0
genre|acousmatic                         0.0
genre|acoustic_blues                     0.0
genre|acoustic_chill                     0.0
genre|acoustic_cover                     0.0
genre|acoustic_guitar_cover              0.0
genre|acoustic_pop                       0.0
genre|acou

#### Normatization (MinMaxScalar)
Finally, we must normalize a few variables. As can be seen in the example below, the popularity variables are not normalized to the range of 0 to 1, which would cause issues with the consine similarity function. Furthermore, the audio features are not adjusted

In [19]:
# Normalization
artist_popularity = song_df[["artist_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
popularity_scaled = pd.DataFrame(scaler.fit_transform(artist_popularity), columns = artist_popularity.columns)
popularity_scaled.head()

,artist_pop
0,0.60
1,0.60
2,0.75
3,0.75
4,0.75


we use the following cell to construct all the features described above and combine all the variables into a new dataframe

In [20]:
# paramter: (df: dataframe, float_columns: dtype float columns)
# return: final feature dataset
def create_feature_set(df, float_columns):
    # Applying tfidf on genre list to most prevalent genre
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)
    # Applying Sentiment analysis
    df = sentiment_analysis(df, "track_name")
    # One-hot Encoding
    ohe_subject = one_hot_encoding(df, 'subjectivity', 'subject') * 0.3
    ohe_polarity = one_hot_encoding(df, 'polarity','polar') * 0.5
    ohe_key_feature = one_hot_encoding(df, 'key','key') * 0.5
    ohe_mode_feature = one_hot_encoding(df, 'mode','mode') * 0.5
    # Scaling popularity columns
    track_artist_popularity = df[["artist_pop", "track_pop"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    popularity_scaled = pd.DataFrame(scaler.fit_transform(track_artist_popularity), columns = track_artist_popularity.columns) * 0.2 
    # Scaling audio features (danceability, liveness..)
    floats = df[float_columns].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2
    # Concanenate all features
    feature_set = pd.concat([genre_df, floats_scaled, popularity_scaled, ohe_subject, ohe_polarity, ohe_key_feature, ohe_mode_feature], axis = 1)
    feature_set['id'] = df['track_uri'].values
    return feature_set

In [21]:
# Save the data and generate the features
float_columns = song_df.dtypes[song_df.dtypes == 'float64'].index.values
song_df.to_csv("songs_data3.csv", index = False)

In [22]:
# Generate features
complete_feature_set = create_feature_set(song_df, float_columns=float_columns)
complete_feature_set.to_csv("complete_feature_data5.csv", index = False)

In [23]:
complete_feature_set.head()

genre|21st_century_classical  genre|432hz  genre|5th_wave_emo  \
0                           0.0          0.0                 0.0   
1                           0.0          0.0                 0.0   
2                           0.0          0.0                 0.0   
3                           0.0          0.0                 0.0   
4                           0.0          0.0                 0.0   

   genre|_hip_hop  genre|_house  genre|_roll  genre|a_cappella  \
0             0.0           0.0          0.0               0.0   
1             0.0           0.0          0.0               0.0   
2             0.0           0.0          0.0               0.0   
3             0.0           0.0          0.0               0.0   
4             0.0           0.0          0.0               0.0   

   genre|aarhus_indie  genre|abstract  genre|abstract_beats  \
0                 0.0             0.0                   0.0   
1                 0.0             0.0                   0.0   
2                 0.0             0.0                   0.0   
3                 0.0             0.0                   0.0   
4                 0.0             0.0                   0.0   

   genre|abstract_hip_hop  genre|abstract_idm  genre|accordion  \
0                     0.0                 0.0              0.0   
1                     0.0                 0.0              0.0   
2                     0.0                 0.0              0.0   
3                     0.0                 0.0              0.0   
4                     0.0                 0.0              0.0   

   genre|acid_house  genre|acid_jazz  genre|acid_rock  genre|acousmatic  \
0               0.0              0.0              0.0               0.0   
1               0.0              0.0              0.0               0.0   
2               0.0              0.0              0.0               0.0   
3               0.0              0.0              0.0               0.0   
4               0.0              0.0              0.0               0.0   

   genre|acoustic_blues  genre|acoustic_chill  genre|acoustic_cover  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   genre|acoustic_guitar_cover  genre|acoustic_pop  genre|acoustic_punk  \
0                          0.0                 0.0                  0.0   
1                          0.0                 0.0                  0.0   
2                          0.0                 0.0                  0.0   
3                          0.0                 0.0                  0.0   
4                          0.0                 0.0                  0.0   

   genre|acoustic_rock  genre|action_rock  genre|adoracion  \
0                  0.0                0.0              0.0   
1                  0.0                0.0              0.0   
2                  0.0                0.0              0.0   
3                  0.0                0.0              0.0   
4                  0.0                0.0              0.0   

   genre|adoracion_pentecostal  genre|adult_standards  genre|adventista  \
0                          0.0                    0.0               0.0   
1                          0.0                    0.0               0.0   
2                          0.0                    0.0               0.0   
3                          0.0                    0.0               0.0   
4                          0.0                    0.0               0.0   

   genre|aesthetic_rap  genre|afghan_pop  genre|african  genre|african_gospel  \
0                  0.0               0.0            0.0                   0.0   
1                  0.0               0.0            0.0                   0.0   
2                  0.0   

### Content Based Filtering Recommendation

The following step is to carry out content-based filtering using the available song features. We combine all of the songs on a playlist into a single summary vector to achieve this. Then, we compare the summarized playlist vector to every song (excluding the songs in the playlist) in the database to see how similar they are. The most pertinent music that isn't on the playlist is then found using the similarity measure, and we recommend it

In this part, there are three steps:

 - Select a playlist: This section involves retrieving a playlist.
 - Features extraction:  We retire both playlist-of-interest features and non-playlist-of-interest features in this section.
 - Find similarities: Here, we contrast the features of the summary playlist with those of all other songs.

In [25]:
# test dataset
test_df = df[df['name'] == "Mom's playlist"]
test_df.to_csv("test_df1.csv")

In [26]:
# Convert users playlist into single vector
# parameter: (feature_set: dataframe of spotify songs, df: playlist dataframe)
# return: (single vector feature of playlist)
def generate_feature(feature_set, df):
    # features in the playlist
    feature_set_playlist = feature_set[feature_set['id'].isin(df['id'].values)]
    # non-playlist song features
    feature_set_nonplaylist = feature_set[~feature_set['id'].isin(df['id'].values)]
    final_feature_set_playlist = feature_set_playlist.drop(columns = "id")
    return final_feature_set_playlist.sum(axis = 0), feature_set_nonplaylist

In [27]:
# Generate the features
feature_set_vector, feature_set_nonplaylist = generate_feature(complete_feature_set, test_df)

In [28]:
# Non-playlist features
feature_set_nonplaylist.head()

genre|21st_century_classical  genre|432hz  genre|5th_wave_emo  \
0                           0.0          0.0                 0.0   
1                           0.0          0.0                 0.0   
2                           0.0          0.0                 0.0   
3                           0.0          0.0                 0.0   
4                           0.0          0.0                 0.0   

   genre|_hip_hop  genre|_house  genre|_roll  genre|a_cappella  \
0             0.0           0.0          0.0               0.0   
1             0.0           0.0          0.0               0.0   
2             0.0           0.0          0.0               0.0   
3             0.0           0.0          0.0               0.0   
4             0.0           0.0          0.0               0.0   

   genre|aarhus_indie  genre|abstract  genre|abstract_beats  \
0                 0.0             0.0                   0.0   
1                 0.0             0.0                   0.0   
2                 0.0             0.0                   0.0   
3                 0.0             0.0                   0.0   
4                 0.0             0.0                   0.0   

   genre|abstract_hip_hop  genre|abstract_idm  genre|accordion  \
0                     0.0                 0.0              0.0   
1                     0.0                 0.0              0.0   
2                     0.0                 0.0              0.0   
3                     0.0                 0.0              0.0   
4                     0.0                 0.0              0.0   

   genre|acid_house  genre|acid_jazz  genre|acid_rock  genre|acousmatic  \
0               0.0              0.0              0.0               0.0   
1               0.0              0.0              0.0               0.0   
2               0.0              0.0              0.0               0.0   
3               0.0              0.0              0.0               0.0   
4               0.0              0.0              0.0               0.0   

   genre|acoustic_blues  genre|acoustic_chill  genre|acoustic_cover  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   genre|acoustic_guitar_cover  genre|acoustic_pop  genre|acoustic_punk  \
0                          0.0                 0.0                  0.0   
1                          0.0                 0.0                  0.0   
2                          0.0                 0.0                  0.0   
3                          0.0                 0.0                  0.0   
4                          0.0                 0.0                  0.0   

   genre|acoustic_rock  genre|action_rock  genre|adoracion  \
0                  0.0                0.0              0.0   
1                  0.0                0.0              0.0   
2                  0.0                0.0              0.0   
3                  0.0                0.0              0.0   
4                  0.0                0.0              0.0   

   genre|adoracion_pentecostal  genre|adult_standards  genre|adventista  \
0                          0.0                    0.0               0.0   
1                          0.0                    0.0               0.0   
2                          0.0                    0.0               0.0   
3                          0.0                    0.0               0.0   
4                          0.0                    0.0               0.0   

   genre|aesthetic_rap  genre|afghan_pop  genre|african  genre|african_gospel  \
0                  0.0               0.0            0.0                   0.0   
1                  0.0               0.0            0.0                   0.0   
2                  0.0   

In [29]:
# Summarized playlist features
feature_set_vector

genre|21st_century_classical             0.000000
genre|432hz                              0.000000
genre|5th_wave_emo                       0.000000
genre|_hip_hop                           0.000000
genre|_house                             0.000000
genre|_roll                              0.000000
genre|a_cappella                         0.000000
genre|aarhus_indie                       0.000000
genre|abstract                           0.000000
genre|abstract_beats                     0.000000
genre|abstract_hip_hop                   0.000000
genre|abstract_idm                       0.000000
genre|accordion                          0.000000
genre|acid_house                         0.000000
genre|acid_jazz                          0.000000
genre|acid_rock                          0.000000
genre|acousmatic                         0.000000
genre|acoustic_blues                     0.000000
genre|acoustic_chill                     0.000000
genre|acoustic_cover                     0.000000


#### Cosine Similarity

Finding similarities between the summarized playlist vector and all other tracks is the final puzzle

Since we are performing the cosine similarity measure between each row vector (song) and the column vector of the summarized playlist feature, one significant advantage of doing this is that the time complexity of the entire algorithm is identical to a matrix multiplication

In [30]:
# parameter: (df: dataframe, features: single playlist vector, non playlist features: songs not in the playlist)
# return: Top recommendation
def generate_recommendation(df, features, non_playlist_features):    
    df_non_playlist = df[df['track_uri'].isin(non_playlist_features['id'].values)]
    # Compare the cosine similarity of the playlist and the entire collection of songs
    df_non_playlist['sim'] = cosine_similarity(non_playlist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    top_non_playlist_df = df_non_playlist.sort_values('sim', ascending = False).head(40)
    top_non_playlist_df['url'] = top_non_playlist_df['track_uri'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    return top_non_playlist_df

In [31]:
# Genreate top recommendation
recommend = generate_recommendation(song_df, feature_set_vector, feature_set_nonplaylist)
recommend.head()

C:\Users\Checkout\AppData\Local\Temp\ipykernel_9904\2672588807.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_playlist['sim'] = cosine_similarity(non_playlist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_uri,uri,track_href,analysis_url,duration_ms_x,time_signature,pos,artist_name,artist_uri,track_name,album_uri,duration_ms_y,album_name,name,artist_pop,genres,track_pop,id,artists_song,genres_list,subjectivity,polarity,sim,url
3030,0.807,0.887,1.0,-3.892,1.0,0.2750,0.00381,0.000000,0.3910,0.780,160.517,audio_features,1lItf5ZXJc1by9SbPeljFd,spotify:track:1lItf5ZXJc1by9SbPeljFd,https://api.spotify.com/v1/tracks/1lItf5ZXJc1b...,https://api.spotify.com/v1/audio-analysis/1lIt...,144244.0,4.0,2,Zay Hilfigerrr,spotify:artist:7nKeLE1toRtW4M279iS26h,Juju On That Beat (TZ Anthem),spotify:album:5sQTfUX9uSwfCtPiYGO20V,144244,Juju On That Beat (TZ Anthem),Hype,41,unknown,59,1lItf5ZXJc1by9SbPeljFd,Zay HilfigerrrJuju On That Beat (TZ Anthem),[unknown],low,Neutral,0.838339,https://i.scdn.co/image/ab67616d00001e02186678...
6439,0.943,0.949,1.0,-4.220,1.0,0.2220,0.03620,0.000000,0.4170,0.701,110.984,audio_features,5BtN3kFKoISSWFpXWz7O4L,spotify:track:5BtN3kFKoISSWFpXWz7O4L,https://api.spotify.com/v1/tracks/5BtN3kFKoISS...,https://api.spotify.com/v1/audio-analysis/5BtN...,199432.0,4.0,0,QUE.,spotify:artist:4Mop3hBjHoSoKwwkzKD3Gi,OG Bobby Johnson,spotify:album:6hepURJaNyoswkaxgSQzd0,199432,OG Bobby Johnson,NaN,35,unknown,49,5BtN3kFKoISSWFpXWz7O4L,QUE.OG Bobby Johnson,[unknown],low,Neutral,0.837374,https://i.scdn.co/image/ab67616d00001e02e86d40...
30245,0.612,0.865,1.0,-3.739,1.0,0.0635,0.09820,0.000163,0.0835,0.772,135.990,audio_features,4QKhhlgwDnXBwk23S0dzJd,spotify:track:4QKhhlgwDnXBwk23S0dzJd,https://api.spotify.com/v1/tracks/4QKhhlgwDnXB...,https://api.spotify.com/v1/audio-analysis/4QKh...,223707.0,4.0,24,R. Kelly,spotify:artist:2mxe0TnaNL039ysAj51xPQ,Ignition - Viceroy Remix,spotify:album:1HHUkEqNzuhRcGX2SC2kYd,223707,Ignition,september,66,unknown,32,4QKhhlgwDnXBwk23S0dzJd,R. KellyIgnition - Viceroy Remix,[unknown],low,Neutral,0.836776,https://i.scdn.co/image/ab67616d00001e02272451...
65783,0.819,0.886,1.0,-4.037,1.0,0.2040,0.13000,0.000000,0.7030,0.818,109.976,audio_features,1Wt3EQx1bI0qXpv9pHeTKS,spotify:track:1Wt3EQx1bI0qXpv9pHeTKS,https://api.spotify.com/v1/tracks/1Wt3EQx1bI0q...,https://api.spotify.com/v1/audio-analysis/1Wt3...,275574.0,4.0,3,Open Minded,spotify:artist:6BVJACZ8dWsEsUUri3Gc9w,Wazer Wifle!!,spotify:album:5n3IEj8PWi3D0wAOmbHFdn,275574,Wazer Wifle!!,The Collection,27,unknown,38,1Wt3EQx1bI0qXpv9pHeTKS,Open MindedWazer Wifle!!,[unknown],low,Neutral,0.835947,https://i.scdn.co/image/ab67616d00001e02831331...
61488,0.460,0.967,1.0,-3.938,1.0,0.0553,0.00380,0.000002,0.1260,0.925,179.972,audio_features,1Cd4I0Ciq9M3djZr2R9Ztn,spotify:track:1Cd4I0Ciq9M3djZr2R9Ztn,https://api.spotify.com/v1/tracks/1Cd4I0Ciq9M3...,https://api.spotify.com/v1/audio-analysis/1Cd4...,201893.0,4.0,140,K.I.D,spotify:artist:6SOM2ZHlhhImE1yxlwfEe9,Taker,spotify:album:4MCDTJ3g2S0lfxNb5UnF20,201893,Taker,part,25,unknown,22,1Cd4I0Ciq9M3djZr2R9Ztn,K.I.DTaker,[unknown],low,Neutral,0.835822,https://i.scdn.co/image/ab67616d00001e02f97baa...
